In [3]:
from collections import defaultdict, Counter
import math
import pprint

In [4]:
sentences = [
    "The_DET cat_NOUN sleeps_VERB",
    "A_DET dog_NOUN barks_VERB",
    "The_DET dog_NOUN sleeps_VERB",
    "My_DET dog_NOUN runs_VERB fast_ADV",
    "A_DET cat_NOUN meows_VERB loudly_ADV",
    "Your_DET cat_NOUN runs_VERB",
    "The_DET bird_NOUN sings_VERB sweetly_ADV",
    "A_DET bird_NOUN chirps_VERB"
]

In [5]:
transition_counts = defaultdict(Counter)
emission_counts = defaultdict(Counter)
tag_counts = Counter()

In [6]:
for sentence in sentences:
    tokens = sentence.split()
    prev_tag = "<START>"
    for token in tokens:
        word, tag = token.rsplit("_", 1)
        emission_counts[tag][word] += 1
        tag_counts[tag] += 1
        transition_counts[prev_tag][tag] += 1
        prev_tag = tag
    transition_counts[prev_tag]["<END>"] += 1

In [7]:
def normalize(counter):
    total = sum(counter.values())
    return {k: v / total for k, v in counter.items()}

In [8]:
transition_probs = {tag: normalize(next_tags) for tag, next_tags in transition_counts.items()}
emission_probs = {tag: normalize(words) for tag, words in emission_counts.items()}
states = list(emission_probs.keys())

In [9]:
def viterbi(words, states, emission_probs, transition_probs):
    V = [{}]
    path = {}

    for state in states:
        emit_p = emission_probs[state].get(words[0], 1e-6)
        trans_p = transition_probs["<START>"].get(state, 1e-6)
        V[0][state] = math.log(trans_p) + math.log(emit_p)
        path[state] = [state]

    for t in range(1, len(words)):
        V.append({})
        new_path = {}

        for curr_state in states:
            emit_p = emission_probs[curr_state].get(words[t], 1e-6)
            (prob, prev_state) = max(
                (V[t-1][y0] + math.log(transition_probs[y0].get(curr_state, 1e-6)) + math.log(emit_p), y0)
                for y0 in states
            )
            V[t][curr_state] = prob
            new_path[curr_state] = path[prev_state] + [curr_state]
        path = new_path

    n = len(words) - 1
    (prob, final_state) = max(
        (V[n][y] + math.log(transition_probs[y].get("<END>", 1e-6)), y) for y in states
    )
    return path[final_state]

In [10]:
test_sentences = [
    "The cat meows",
    "My dog barks loudly"
]

In [11]:
for sentence in test_sentences:
    words = sentence.split()
    tags = viterbi(words, states, emission_probs, transition_probs)
    print(f"Sentence: {sentence}")
    print("POS Tags: {tags}")
    print()

Sentence: The cat meows
POS Tags: {tags}

Sentence: My dog barks loudly
POS Tags: {tags}

